# A code to correct any systemmatic flux density offsets in radio images.
First run TraP on the images with a high detection threshold to get bright sources.
Then run this script to obtain the multiplicative flux density correction for each individual image.
It compares the extracted flux density to the average flux density for each source, by fitting a straight line between the datapoints, the gradient gives the multiplicative flux density factor.
The key assumption is that the majority of sources are stable.

Advice for use
 - select bright, point-like sources within the FWHM of the beam.
 - Use the TraP monitoring list capability (with a very high detection threshold) to only get the sources chosen

 note - only process one frequency at once

In [1]:
import scipy as sp
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import *
from sqlalchemy.orm import relationship
import tkp.db
import sys
from dblogin import * # This file contains all the variables required to connect to the database
import matplotlib
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['legend.loc'] = 'best'
from matplotlib.ticker import NullFormatter
from matplotlib.font_manager import FontProperties
from astropy.io import fits


In [23]:
# The input database, dataset and thresholds
dataset_id = 43
database = 'antoniar'
plots=False
saveFigUrl='/scratch/antoniar/ELAIS_flxcorr/'

In [24]:
# Connect to the database and run the queries
db = tkp.db.Database(engine=engine, host=host, port=port, user=user, password=password, database=database)
db.connect()
session = db.Session()

Not configuring pre-configured database


In [25]:
# Obtain the average flux density measurement for each source
# RuncatID, Avg Int Flux Density
from tkp.db.model import Runningcatalog
from tkp.db.model import RunningcatalogFlux

Runcats = session.query(Runningcatalog,RunningcatalogFlux).select_from(join(Runningcatalog,RunningcatalogFlux)).filter(Runningcatalog.dataset_id == dataset_id).all()
RuncatsData=[[Runcats[i].Runningcatalog.id,Runcats[i].RunningcatalogFlux.avg_f_int,Runcats[i].Runningcatalog.datapoints] for i in range(len(Runcats))]
RuncatsData = pd.DataFrame(data=RuncatsData, columns=['RuncatID','avg_f_int','datapoints'])
print(RuncatsData)

     RuncatID  avg_f_int  datapoints
0       72678   0.006501          50
1       72517   0.020591          52
2       72581   0.004766          51
3       72648   0.004641          51
4       72736   0.208109          60
..        ...        ...         ...
396     72458   0.047843          60
397     72622   0.010380          59
398     72650   0.026305          60
399     72530   0.040964          60
400     72608   0.031722          60

[401 rows x 3 columns]


In [26]:
# Get all the extracted sources with their runcat ids

from tkp.db.model import Assocxtrsource

listRuncats = RuncatsData.RuncatID
listRuncats = list(listRuncats)
listRuncats.sort()
AssocSrcs = session.query(Assocxtrsource).filter(Assocxtrsource.runcat_id.in_(listRuncats)).all()
AssocSrcs = [[AssocSrcs[i].xtrsrc.id, AssocSrcs[i].runcat.id] for i in range(len(AssocSrcs))]
AssocSrcs = pd.DataFrame(data=AssocSrcs, columns=['xtrsrc','RuncatID'])
print(AssocSrcs)

        xtrsrc  RuncatID
0      1526669     72440
1      1526670     72441
2      1526671     72442
3      1526672     72443
4      1526673     72444
...        ...       ...
21719  1548285     72809
21720  1548286     72811
21721  1548287     72818
21722  1548288     72819
21723  1548289     72833

[21724 rows x 2 columns]


In [27]:
# Loop through the images

from tkp.db.model import Image

Images = session.query(Image).filter(Image.dataset_id == dataset_id).all()
numImgs=len(Images)

print(numImgs)

72


In [32]:
# For each image obtain the extracted source flux density for each RuncatID
    
from tkp.db.model import Extractedsource

for a in range(len(Images)):
    extractedSrcs = session.query(Extractedsource).filter(Extractedsource.image_id == Images[a].id).all()
    extractedSrcsData = [[extractedSrcs[b].id,extractedSrcs[b].f_int,extractedSrcs[b].f_int_err] for b in range(len(extractedSrcs))]
    extractedSrcsData = pd.DataFrame(data=extractedSrcsData, columns=['xtrsrc','f_int','f_int_err'])
    extractedSrcsData = pd.merge(extractedSrcsData,AssocSrcs, on='xtrsrc')
    plotData = pd.merge(extractedSrcsData,RuncatsData,on='RuncatID')

    # Fit straight line through the origin and output the gradient
    x=np.array(plotData.avg_f_int)
    y=np.array(plotData.f_int)

    if len(x) > 0:
        x = x[:,np.newaxis]
        b, _, _, _ = np.linalg.lstsq(x,y,rcond=None) # fit a straight line going through the origin   
        print(Images[a].url, b[0])
        xrng = np.linspace(0,max(plotData.avg_f_int),10)
        xfit = [x*b for x in xrng]
        if plots==True:
            # Plot scatter graph of data points
            plt.errorbar(plotData.avg_f_int, plotData.f_int, yerr=plotData.f_int_err, fmt='o')
            plt.plot(xrng,xfit,'-')
            plt.show()
        
        #Open each image and multiply the pixel values by the gradient - save each image to new location
        hdu = fits.open(Images[a].url)
        data = hdu[0].data[0][0]
        hdu[0].data[0][0] = data / b[0]
        newFilename = Images[a].url.split('/')[-1]
        newFilename = saveFigUrl+'corrected_'+newFilename
        hdu.writeto(newFilename)


/scratch/ideruiter/deep_fields_individual_epochs/L229064_image_full_ampphase_di_m.NS.app.restored.fits 1.1735603865775592
/scratch/ideruiter/deep_fields_individual_epochs/L229312_image_full_ampphase_di_m.NS.app.restored.fits 1.1684889467869497
/scratch/ideruiter/deep_fields_individual_epochs/L229387_image_full_ampphase_di_m.NS.app.restored.fits 1.156320814786933
/scratch/ideruiter/deep_fields_individual_epochs/L229673_image_full_ampphase_di_m.NS.app.restored.fits 1.1669130024214924
/scratch/ideruiter/deep_fields_individual_epochs/L230461_image_full_ampphase_di_m.NS.app.restored.fits 1.1728230623725007
/scratch/ideruiter/deep_fields_individual_epochs/L230779_image_full_ampphase_di_m.NS.app.restored.fits 1.1561814130377934
/scratch/ideruiter/deep_fields_individual_epochs/L231211_image_full_ampphase_di_m.NS.app.restored.fits 1.1440901256561926
/scratch/ideruiter/deep_fields_individual_epochs/L231505_image_full_ampphase_di_m.NS.app.restored.fits 1.0918544478112528
/scratch/ideruiter/deep_f